In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import gc
import os
import sys
import calendar
%matplotlib inline

In [ ]:
!wget https://www.dropbox.com/s/brixkogrmhan6ed/cab_rides.csv

In [ ]:
!wget https://www.dropbox.com/s/ncqb2ctkg7da11k/weather.csv

In [ ]:
#memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64) 
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df        

In [ ]:
cab_data = pd.read_csv("cab_rides.csv")
cab_data = reduce_mem_usage(cab_data)
weather_data = pd.read_csv("weather.csv")
weather_data = reduce_mem_usage(weather_data)

In [ ]:
cab_data

In [ ]:
cab_data.head()

In [ ]:
weather_data.columns

In [ ]:
import datetime
cab_data['datetime'] = pd.to_datetime(cab_data['time_stamp'])
cab_data
weather_data['date_time'] = pd.to_datetime(weather_data['time_stamp'])

In [ ]:
cab_data.columns

In [ ]:
cab_data.shape #no of rows and columns

In [ ]:
weather_data.shape

In [ ]:
cab_data.describe()

In [ ]:
weather_data.describe()

In [ ]:
a = pd.concat([cab_data, weather_data])  #Concatenate/merging two data frames

In [ ]:
a.head()

In [ ]:
a['day'] = a.date_time.dt.day #setting day
a['hour'] = a.date_time.dt.hour #setting hour

In [ ]:
a.tail()

In [ ]:
a.fillna(0,inplace=True) #replacing NaN values with 0

In [ ]:
a.tail()

In [ ]:
a.columns

In [ ]:
a.groupby('cab_type').count() #group data by cab type count returns the count of how many times a given object occurs in a List.

In [ ]:
a.groupby('cab_type').count().plot(kind="bar", figsize=(10,5)) #or count().plot.bar()

In [ ]:
a['price'].value_counts().plot(kind='bar', figsize=(100, 50), color='blue')

In [ ]:
a['hour'].value_counts().plot(kind='bar', figsize=(10,5), color='green')

In [ ]:
import matplotlib.pyplot as plt
x = a['hour']
y = a['price']
plt.plot(x,y)
plt.show() #how hour varies along price

In [ ]:
x = a['rain']
y = a['price']
plt.plot(x,y)
plt.show()

In [ ]:
a.columns

In [ ]:
x1 = a[['distance', 'temp', 'pressure', 'humidity', 'wind', 'rain', 'day', 'hour', 'surge_multiplier', 'clouds']]
y1 = a['price']
#attributes used to train model

In [ ]:
# Using Skicit=learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
x_train, y_train, x_test, y_test = train_test_split(x1, y1, test_size=0.25, random_state=42)

In [ ]:
linear = LinearRegression()
linear.fit(x_train, x_test)

In [ ]:
predictions = linear.predict(y_train) #finding predictions
predictions

In [ ]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': predictions}) #determining actual and predicted values
df

In [ ]:
df1 = df.head(25)
df1.plot(kind='bar', figsize=(26,10))
plt.grid(which='major', linestyle = '-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth = '0.5', color='black')
plt.show() #comparision of actual and predicted values